In [3]:
pip install Cython --install-option="--no-cython-compile"

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: --install-option is deprecated because it forces pip to use the 'setup.py install' command which is itself deprecated. pip 23.1 will enforce this behaviour change. A possible replacement is to use --config-settings. Discussion can be found at https://github.com/pypa/pip/issues/11358
DEPRECATION: --no-binary currently disables reading from the cache of locally built wheels. In the future --no-binary will not influence the wheel cache. pip 23.1 will enforce this behaviour change. A possible replacement is to use the --no-cache-dir option. You can use the flag --use-feature=no-binary-enable-wheel-cache to test the upcoming behaviour. Discussion can be found at https://github.com/pypa/pip/issues/11453


In [9]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from Cython.Build.Inline import _get_build_extension
import seaborn as sns
import tensorflow as tf
import keras


In [23]:
# Load the data from a CSV file
df = pd.read_excel('tata power.xlsx')
df


,Date,Price,Open,High,Low,Volume,MA,MA_7_Close,MV_7 Open,MV_7 High,MV_7 low,MA_14_Close,MV_14 Open,MV_14 High,MV_14 Low,MA_21_Close,MV_21 open,MV_21 High,MV_21 Low
0,"Apr 03, 2023",193.95,193.35,196.05,192.05,14.93M,0.0194,200.10,NaN,NaN,NaN,202.950,NaN,NaN,NaN,207.75,NaN,NaN,NaN
1,"Mar 31, 2023",190.25,188.50,191.60,187.95,13.52M,0.0256,190.45,NaN,NaN,NaN,198.675,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Mar 29, 2023",185.50,184.50,187.70,183.30,17.48M,0.0084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Mar 28, 2023",183.95,186.70,187.50,182.35,12.02M,-0.0145,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Mar 27, 2023",186.65,192.90,194.00,184.45,22.25M,-0.0316,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1481,"Apr 11, 2017",85.45,87.45,90.15,81.15,32.22M,-0.0184,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1482,"Apr 10, 2017",87.05,87.80,87.85,86.85,5.51M,-0.0051,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1483,"Apr 07, 2017",87.50,87.00,88.15,86.20,5.37M,0.0023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1484,"Apr 06, 2017",87.30,88.10,88.20,86.65,4.47M,-0.0102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
df.head()

,Date,Price,Open,High,Low,Volume,MA,MA_7_Close,MV_7 Open,MV_7 High,MV_7 low,MA_14_Close,MV_14 Open,MV_14 High,MV_14 Low,MA_21_Close,MV_21 open,MV_21 High,MV_21 Low
0,"Apr 03, 2023",193.95,193.35,196.05,192.05,14.93M,0.0194,200.10,NaN,NaN,NaN,202.950,NaN,NaN,NaN,207.75,NaN,NaN,NaN
1,"Mar 31, 2023",190.25,188.50,191.60,187.95,13.52M,0.0256,190.45,NaN,NaN,NaN,198.675,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Mar 29, 2023",185.50,184.50,187.70,183.30,17.48M,0.0084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Mar 28, 2023",183.95,186.70,187.50,182.35,12.02M,-0.0145,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Mar 27, 2023",186.65,192.90,194.00,184.45,22.25M,-0.0316,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
# Clean the data by removing any rows with missing values
df = df.dropna()

In [34]:
# Feature engineering: add technical indicators
df['MA_7'] = df['Close'].rolling(window=7).mean()
df['MA_14'] = df['Close'].rolling(window=14).mean()
df['MA_21'] = df['Close'].rolling(window=21).mean()
df['EMA_7'] = df['Close'].ewm(span=7, adjust=False).mean()
df['EMA_14'] = df['Close'].ewm(span=14, adjust=False).mean()
df['EMA_21'] = df['Close'].ewm(span=21, adjust=False).mean()

KeyError: 'Close'

In [36]:
df.head()


,Date,Price,Open,High,Low,Volume,MA,MA_7_Close,MV_7 Open,MV_7 High,...,MV_14 Open,MV_14 High,MV_14 Low,MA_21_Close,MV_21 open,MV_21 High,MV_21 Low,MA_7,MA_14,MA_21


In [27]:
# Split the data into training, validation, and testing sets
train, test = train_test_split(df, test_size=0.2, random_state=42)
train, val = train_test_split(train, test_size=0.2, random_state=42)


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
# Create the feature matrix X and the target vector y for each set
X_train = train[['Open', 'High', 'Low', 'MA_7', 'MA_14', 'MA_21', 'EMA_7', 'EMA_14', 'EMA_21']]
y_train = train['Close']
X_val = val[['Open', 'High', 'Low', 'MA_7', 'MA_14', 'MA_21', 'EMA_7', 'EMA_14', 'EMA_21']]
y_val = val['Close']
X_test = test[['Open', 'High', 'Low', 'MA_7', 'MA_14', 'MA_21', 'EMA_7', 'EMA_14', 'EMA_21']]
y_test = test['Close']


In [ ]:
# Normalize the data using MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df['Open'] = scaler.fit_transform(df[['Open']])
df['High'] = scaler.fit_transform(df[['High']])
df['Low'] = scaler.fit_transform(df[['Low']])
df['Close'] = scaler.fit_transform(df[['Close']])

In [ ]:
# Train a Random Forest Regressor model
model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
model.fit(X_train, y_train)

In [ ]:
# Evaluate the model on the validation set
y_val_pred = model.predict(X_val)
mse = mean_squared_error(y_val, y_val_pred)
print('Mean Squared Error on Validation Set:', mse)

In [ ]:
# Test the model on the testing set
y_test_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_test_pred)
print('Mean Squared Error on Testing Set:', mse)

In [ ]:
# Plot the predicted and actual prices for the testing set
plt.plot(y_test_pred, label='Predicted')
plt.plot(y_test.values, label='Actual')
plt.legend()
plt.show()

In [ ]:
# Save the model to a file
import joblib
joblib.dump(model, 'stock_model.joblib')

In [ ]:

# Load the model from the file and use it to make predictions
model = joblib.load('stock_model.joblib')
X_new = [[0.5, 0.6, 0.4]]
y_new_pred = model.predict(X_new)
print('Predicted Close Price:', y_new_pred)